In [1]:
# To run go to Kernel -> Restart and run all
#ALL IMPORTS
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# READING DATA
season13_14 = pd.read_csv("13_14.csv")
season14_15 = pd.read_csv("14_15.csv")
season15_16 = pd.read_csv("15_16.csv")
season16_17 = pd.read_csv("16_17.csv")
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")
season10_11 = pd.read_csv("10_11.csv")
season11_12 = pd.read_csv("11_12.csv")
season12_13 = pd.read_csv("12_13.csv")

season14_15 = season14_15.dropna()

In [3]:
# Football class
class Football:
    # Initializer
    def __init__(self, data, name):
        self.data=data
        self.cleanData()
        self.name=name
        self.teams = set(self.data.HomeTeam) | set(self.data.AwayTeam)
        self.referees = set(self.data.Referee)
#         self.teams = {i for i in self.data.HomeTeam}
#         self.teams.update({i for i in self.data.AwayTeam})
    
    # Getting teams
    def getTeams(self):
        return self.teams
    
    # Getting team wins
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams: # populating dict
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def get5TeamWinsHome(self,teams):
        goals = dict()
        for i in teams: # populating dict
            goals[i]=0
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'H' and self.data.HomeTeam[i] in teams:
                goals[self.data.HomeTeam[i]]+=1
        return goals
            
    def get5TeamWinsAway(self,teams):
        goals = dict()
        for i in teams: # populating dict
            goals[i]=0
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A' and self.data.AwayTeam[i] in teams:
                goals[self.data.AwayTeam[i]]+=1
        return goals
    
    def get5TeamLosesHome(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A' and self.data.HomeTeam[i] in teams:
                team[self.data.HomeTeam[i]]+=1
        return team
            
    def get5TeamLosesAway(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'H' and self.data.AwayTeam[i] in teams:
                team[self.data.AwayTeam[i]]+=1
        return team
    
    def get5TeamDrawsHome(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D' and self.data.HomeTeam[i] in teams:
                team[self.data.HomeTeam[i]]+=1
        return team
    def get5TeamDrawsAway(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D' and self.data.AwayTeam[i] in teams:
                team[self.data.AwayTeam[i]]+=1
        return team
    
    def get5TeamGoalsHome(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.HomeTeam[i] in teams:
                team[self.data.HomeTeam[i]]+=int(self.data.FTHG[i])
        return team
    def get5TeamGoalsAway(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.AwayTeam[i] in teams:
                team[self.data.AwayTeam[i]]+=int(self.data.FTAG[i])
        return team
    
    def get5TeamYellowHome(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.HomeTeam[i] in teams:
                team[self.data.HomeTeam[i]]+=int(self.data.HY[i])
        return team
    def get5TeamYellowAway(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.AwayTeam[i] in teams:
                team[self.data.AwayTeam[i]]+=int(self.data.AY[i])
        return team
    
    def get5TeamRedHome(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.HomeTeam[i] in teams:
                team[self.data.HomeTeam[i]]+=int(self.data.HR[i])
        return team
    def get5TeamRedAway(self,teams):
        team = dict()
        for i in teams: # populating dict
            team[i]=0
        for i in range(len(self.data)):
            if self.data.AwayTeam[i] in teams:
                team[self.data.AwayTeam[i]]+=int(self.data.AR[i])
        return team
    
    # Getting team loses
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams: # populating dict
            loseTeams[i]=0
        
        for i in range(len(self.data)): # looping through
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
    
    # Getting team draws
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams: # populating dict
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamGames(self): #How many games were played by each team
        gamesCount = dict()
        for i in self.teams: # populating dict
            gamesCount[i] = 0
        for i in range(len(self.data)):
            gamesCount[self.data.HomeTeam[i]]+=1
            gamesCount[self.data.AwayTeam[i]]+=1
        return gamesCount
    
    def getTeamPoints(self): # Getting points based their out come
        points = dict()
        #WIN 3 POINTS
        #DRAW 1 POINT
        for i in self.teams: # populating dict
            points[i] = 0
        for i in self.getTeamWins():
            points[i]=self.getTeamWins()[i]*3
        for i in self.getTeamDraws():
            points[i]+=self.getTeamDraws()[i]*1
        return points
        
    
    def getTeamsTotalGoals(self): #Getting team total goals
        teamTotalGoals = dict()
        for i in self.teams: # populating dict
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            teamTotalGoals[self.data.HomeTeam[i]] += self.data.FTHG[i]
            teamTotalGoals[self.data.AwayTeam[i]] += self.data.FTAG[i]
            
        return teamTotalGoals
    
    def cleanData(self): # Cleaning data
        colums = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST",
                 "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "B365H", "B365D", "B365A"] # Fields that are staying
        for col in self.data.columns:
            contains = False
            for j in colums:
                if col == j:
                    contains = True
            if not contains:
                self.data = self.data.drop(col, axis=1)
    
    def getWrongPrediction(self): # Getting total of matches, where the prefered team loses or play draw
        #TODO find matches were odds and match results were not equal
        luckyLoser = dict()
        referees = dict()
        goalsDifference = []
        homeOrAway={"Home":0,"Away":0}
        for i in self.teams: # populating dict
            luckyLoser[i] = 0
        for i in self.data.Referee:
            referees[i]=0
        
        kokku=0
        for i in range(len(self.data)):
            homeOdd = float(self.data.B365H[i])
            drawOdd = float(self.data.B365D[i])
            awayOdd = float(self.data.B365A[i])
            
            homeGoals = int(self.data.FTHG[i])
            awayGoals = int(self.data.FTAG[i])
            
            if homeOdd == awayOdd: #If odds are same, then there is no right prediction
                continue
            if homeOdd < awayOdd and self.data.FTR[i]!='H': #and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON SECOND TEAM")
                luckyLoser[self.data.AwayTeam[i]]+=1
                homeOrAway['Away']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
            elif homeOdd > awayOdd and self.data.FTR[i]!='A':# and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON FIRST TEAM")
                luckyLoser[self.data.HomeTeam[i]]+=1
                homeOrAway['Home']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
        print(self.name)
        print(kokku/len(self.data)*100,"%")
        #print(sorted(luckyLoser.items(), key=lambda x: x[1], reverse=True))
        print("REFEREES")
        print(sorted(referees.items(), key=lambda x: x[1], reverse=True))
        print("Where was wrong prediction",homeOrAway)
        print("Goals difference,",goalsDifference)
        print()
                
    
    def showTabel(self):
        wins=[i for i in self.getTeamWins().values()]
        loses=[i for i in self.getTeamLoses().values()]
        draws=[i for i in self.getTeamDraws().values()]
        points=[i for i in self.getTeamPoints().values()]
        games=[i for i in self.getTeamGames().values()]
        tabel = {'Teams': list(self.teams),
                 'Wins': wins,
                 'Loses': loses,
                 'Draws': draws,
                 'Points': points,
                 'Games': games}
        df = pd.DataFrame(tabel)
        df=df.sort_values(by=['Points'],ascending=False)
        df=df.to_string(index=False)
        print(df)
        
    def refereeTotalPenalties(self): # Getting total cards given by referee
        refereePenalties = dict()
        refereeMatches = self.refereeTotalMatches()
        for i in self.referees: # populating dict
            refereePenalties[i] = 0
        for i in range(len(self.data)):
            refereePenalties[self.data.Referee[i]] += self.data.HY[i] + self.data.AY[i] + self.data.HR[i] + self.data.AR[i]
            
        for i in refereePenalties:
            refereePenalties[i] = round(refereePenalties[i] / refereeMatches[i], 2)
        return refereePenalties
    
    def refereeTotalMatches(self): -# Getting total number of matches on how many referee matches he/she was
        refereeMatches = dict()
        for i in self.referees: # populating dict
            refereeMatches[i] = 0
        for i in range(len(self.data)):
            refereeMatches[self.data.Referee[i]] += 1
        return refereeMatches
    
    def goalsPerSeason(self): # Getting total number of goals for team given season
        teams=dict()
        for i in self.teams: # populating dict
            teams[i] = 0
        for i in range(len(self.data)):
            if int(self.data.FTHG[i])>0:
                teams[self.data.HomeTeam[i]]+=1
            if int(self.data.FTAG[i])>0:
                teams[self.data.AwayTeam[i]]+=1
        return teams
        
        

SyntaxError: invalid syntax (<ipython-input-3-8f5af02a80f9>, line 281)

In [ ]:
season1718 = Football(season17_18,"Season 17_18")
season1819 = Football(season18_19,"Season 18_19")
season1920 = Football(season19_20,"Season 19_20")
#season1718.showTabel()
#season1718.getWrongPrediction()
#season1819.getWrongPrediction()
#season1920.getWrongPrediction()
#print("Refs:\n", season1718.refereeTotalPenalties())

In [ ]:
#CREATING PREDICTION MODEL

predictingData = season19_20[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
predictingData=predictingData.append(season18_19[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season17_18[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season15_16[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season16_17[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season13_14[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season14_15[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season10_11[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season11_12[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season12_13[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)


###############TESTING PURPOSE TO FIND BEST PARAMETERs FOR ACCURACY #############################

'''df = pd.DataFrame(columns=['accuracy', 'n_estimators', 'max_depth'])    
    
for i in range(5,30):
    print(i)
    for j in range(1,int(i*3/5)):
        X_train = pd.get_dummies(predictingData, columns=["HomeTeam","AwayTeam"])
        y_train = season19_20['FTR']
        y_train = y_train.append(season18_19['FTR'])
        y_train = y_train.append(season17_18['FTR'])
        y_train = y_train.append(season15_16['FTR'])
        y_train = y_train.append(season16_17['FTR'])
#         y_train = y_train.append(season13_14['FTR'])
#         y_train = y_train.append(season14_15['FTR'])
#         y_train = y_train.append(season10_11['FTR'])
#         y_train = y_train.append(season11_12['FTR'])
#         y_train = y_train.append(season12_13['FTR'])

        rf = RandomForestClassifier(n_estimators=i, max_depth=j).fit(X_train, y_train)
        x_train, x_test, y_train, y_test = train_test_split(X_train,y_train, train_size=0.75)
        clrTree = rf.fit(x_train, y_train)
        outTree = clrTree.predict(x_test)
        a=pd.DataFrame([[accuracy_score(y_test, outTree)*100,i,j]],columns=['accuracy', 'n_estimators', 'max_depth'])
        df=df.append(a,ignore_index=True)
        #print("Accuracy for Decision Tree Classifier: " + str(accuracy_score(y_test, outTree)*100)+"%"," ",i)'''


In [ ]:
#print(df.sort_values(by=['accuracy'], ascending=False))

# Appending data to dataframe
predictingData = season19_20[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
predictingData=predictingData.append(season18_19[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season17_18[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)

predictingData=predictingData.append(season15_16[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season16_17[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season13_14[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season14_15[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season10_11[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season11_12[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
# predictingData=predictingData.append(season12_13[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)

# Creating prediction model
X_train = pd.get_dummies(predictingData, columns=["HomeTeam","AwayTeam"])
y_train = season19_20['FTR']
y_train = y_train.append(season18_19['FTR'])
y_train = y_train.append(season17_18['FTR'])
                                     
y_train = y_train.append(season15_16['FTR'])
y_train = y_train.append(season16_17['FTR'])
# y_train = y_train.append(season13_14['FTR'])
# y_train = y_train.append(season14_15['FTR'])
# y_train = y_train.append(season10_11['FTR'])
# y_train = y_train.append(season11_12['FTR'])
# y_train = y_train.append(season12_13['FTR'])

                                     
rf = RandomForestClassifier(n_estimators=216, max_depth=4, random_state=0).fit(X_train, y_train) #Random fores
x_train, x_test, y_train, y_test = train_test_split(X_train,y_train, train_size=0.75) # Splitting data
clrTree = rf.fit(x_train, y_train) # Fitting data
outTree = clrTree.predict(x_test) # Testing model with test data
print("Accuracy for Decision Tree Classifier: " + str(accuracy_score(y_test, outTree)*100)+"%")


###### TESTING: GIVEN OUR OWN TEAMS AND ODDS ###############
finishColumns=X_train
finishColumns.drop(finishColumns.index, inplace=True)

row={}
for i in finishColumns:
    if i=='HomeTeam_Aston Villa' or i=='AwayTeam_Southampton': 
        row[i]=1
    elif i=='B365H':
        row[i]=7.36
    elif i=='B365D':
        row[i]=1.02
    elif i=='B365A':
        row[i]=7.35
    else:
        row[i]=0
finishColumns = finishColumns.append(row, ignore_index=True)
print(rf.predict(finishColumns))

In [ ]:
def getScore(homeTeam, awayTeam, x, y, z): # Given home team, away team and their odss
    # x = home team odds
    # y = draw odds
    # z = away team odds
    finishColumns=X_train
    finishColumns.drop(finishColumns.index, inplace=True)

    row={}
    for i in finishColumns:
        if i=='HomeTeam_' + homeTeam or i=='AwayTeam_' + awayTeam: 
            row[i]=1
        elif i=='B365H':
            row[i]=x
        elif i=='B365D':
            row[i]=y
        elif i=='B365A':
            row[i]=z
        else:
            row[i]=0
    finishColumns = finishColumns.append(row, ignore_index=True)
    wins = []
    for i in range(11):
        wins.append(rf.predict(finishColumns))
    return wins # Return list of 11 iteration predictions on who won

def moreWins(wins): # Given list of prediction on who won
    home = 0
    away = 0
    for i in wins:
        if i == "H":
            home +=1
        elif i == "A":
            away += 1
        else:
            away += 0.5
            home += 0.5
    if home > away: # Returns who won based on 11 iterations of prediction model
        return "H"
    else:
        return "A"

In [ ]:
# Predicting on future matches who won
print(moreWins(getScore("Everton", "Arsenal", 2.6, 3.8, 2.65)))
print(moreWins(getScore("Bournemouth", "Burnley", 2.3, 3.55, 3.2)))
print(moreWins(getScore("Aston Villa", "Southampton", 2.4, 3.7, 2.95)))
print(moreWins(getScore("Brighton", "Sheffield United", 2.32, 3.55, 3.55)))
print(moreWins(getScore("Newcastle", "Crystal Palace", 2.65, 3.25, 2.95)))
print(moreWins(getScore("Norwich", "Wolves", 3.75, 3.9, 2.01)))
print(moreWins(getScore("Man City", "Leicester", 1.44, 5.3, 7.2)))

In [ ]:
################### TESTING PURPOSE, DID NOT USE ######################

# teams1 = set(predictingData["HomeTeam"]) or set(predictingData["AwayTeam"])
teams1 = set(season19_20['HomeTeam']) or set(season19_20['AwayTeam'])
teams2 = teams1.copy()
teamScores = dict()
for i in teams1:
    teamScores[i] = 0
    
# Functions that puts every team against each other with equal odds

# for i in teams1:
#     print(i)
#     for j in teams2:
#         if i != j:
#             wins = getScore(i, j)
#             winner = moreWins(wins)
#             if winner == "H":
#                 teamScores[i] += 1
#             elif winner == "A":
#                 teamScores[j] += 1
# print(sorted(teamScores.items(), key=lambda x: x[1], reverse=True))

In [ ]:
# Prinsts previuos block results
teamScores = sorted(teamScores.items(), key=lambda x: x[0], reverse=True)
teamScores = dict(teamScores)
for key, value in teamScores.items():
    print(key + "," + str(value))

In [ ]:
#GETTING DATA HOME AND AWAY
#Man City, Liverpool, Chelsea, Tottenham, Man United
hooaeg1819 = Football(season18_19,"Season 18_19")
print(hooaeg1819.get5TeamWinsHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamWinsAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamLosesHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamLosesAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamDrawsHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamDrawsAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamGoalsHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamGoalsAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamYellowHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamYellowAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamRedHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1819.get5TeamRedAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))

In [ ]:
hooaeg1920 = Football(season19_20,"Season 19_20")
print(hooaeg1920.get5TeamWinsHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamWinsAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamLosesHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamLosesAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamDrawsHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamDrawsAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamGoalsHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamGoalsAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamYellowHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamYellowAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamRedHome(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))
print(hooaeg1920.get5TeamRedAway(["Man City","Liverpool","Chelsea","Tottenham","Man United"]))

In [ ]:
#38 GAMES WERE PLAYED IN SEASON 18-19
#THE NUMBER NEAR TO EACH TEAM SHOWS IN HOW MUCH MATCHES THE TEAM SCORED THE GOAL
print(sorted(season1819.goalsPerSeason().items(), key=lambda x: x[1], reverse=True))
print(sorted(season1920.goalsPerSeason().items(), key=lambda x: x[1], reverse=True))

In [ ]:
#GET ACTUAL TEAM WINS IN 19-20
team = season1920.getTeamWins() # Printing season 19-20 team wins
team = sorted(team.items(), key=lambda x: x[1], reverse=True)
team = dict(team)
for key, value in team.items():
    print(key + "," + str(value))
print()
#HERE I WILL CREATE MODEL WICH WILL TRY PREDICT SAME RESULT ON PREVIOUS MATCHES
# Appending 2015 to 2019 data to dataframe
predictingData = season18_19[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
predictingData=predictingData.append(season17_18[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season16_17[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)
predictingData=predictingData.append(season15_16[["HomeTeam","AwayTeam","B365H","B365D","B365A"]],ignore_index = True)

X_train = pd.get_dummies(predictingData, columns=["HomeTeam","AwayTeam"])

y_train = season18_19['FTR']
y_train = y_train.append(season17_18['FTR'])
y_train = y_train.append(season16_17['FTR'])
y_train = y_train.append(season15_16['FTR'])

# Training model with random forest
model = RandomForestClassifier(n_estimators=216, max_depth=4).fit(X_train, y_train)

winTeams = dict()
for i in season1920.getTeams(): # Creating dict
    winTeams[i]=0

for i in range(len(season19_20)): # Itering through teams and putting the each other. Getting prediction model team wins
    finishColumns=X_train
    finishColumns.drop(finishColumns.index, inplace=True)
    row={}
    home=""
    away=""
    for j in finishColumns:
        H=str('HomeTeam_'+season19_20.HomeTeam[i])
        A=str('AwayTeam_'+season19_20.HomeTeam[i])
        if j==H or j==A:
            row[j]=1
            home=season19_20.HomeTeam[i]
            away=season19_20.AwayTeam[i]
        elif j=='B365H':
            row[j]=float(season19_20.B365H[i])
        elif j=='B365D':
            row[j]=float(season19_20.B365D[i])
        elif j=='B365A':
            row[j]=float(season19_20.B365A[i])
        else:
            row[j]=0
    finishColumns = finishColumns.append(row, ignore_index=True)
    #print(model.predict(finishColumns),home,away)
    result = model.predict(finishColumns)
#     for i in range(11):
#         wins.append(model.predict(finishColumns))
#     result = moreWins(wins)
    if result == 'H' and home!='':
        winTeams[home]+=1
    elif result == 'A' and away!='':
        winTeams[away]+=1 

winTeams = sorted(winTeams.items(), key=lambda x: x[1], reverse=True) # Printing season 19-20 prediction team wins
winTeams = dict(winTeams)
for key, value in winTeams.items():
    print(key + "," + str(value))